In [ ]:
import os
import re
import time
from datetime import datetime, timezone
from pathlib import Path
from dotenv import load_dotenv
import undetected_chromedriver as uc
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
OUTPUT_CSV_FILE = r'data/Facebook_post_urls.csv'
PROFILE_PATH = r'chrome-profile/fb-pipeline-stage1'

GROUP_URLS = [
    "https://www.facebook.com/groups/302468990428489/",
    "https://www.facebook.com/groups/322977734828852/",
    "https://www.facebook.com/groups/812156038944325/",
    "https://www.facebook.com/groups/1472146056424210/",
    "https://www.facebook.com/groups/426467944402414/",
    "https://www.facebook.com/groups/homerentcm/",
    "https://www.facebook.com/groups/509895225859790/",
    "https://www.facebook.com/groups/1299302030158649/",
    "https://www.facebook.com/groups/530492663958652/",
    "https://www.facebook.com/groups/596670275854317/",
    "https://www.facebook.com/groups/cnxre/",
    "https://www.facebook.com/groups/1885263611797363/",
    "https://www.facebook.com/groups/1897854047114064/",
    "https://www.facebook.com/groups/303531690102574/",
    "https://www.facebook.com/groups/1881982752029163/",
    "https://www.facebook.com/groups/568026117396809/",
    "https://www.facebook.com/groups/1928645537294336/",
    "https://www.facebook.com/groups/298821664628156/",
    "https://www.facebook.com/groups/903971886395138/",
    "https://www.facebook.com/groups/142702946428033/members",
    "https://www.facebook.com/groups/250469775470436/",
    "https://www.facebook.com/groups/1873094122912006/",
    "https://www.facebook.com/groups/2083392538672247/",
    "https://www.facebook.com/groups/864193946960728/",
    "https://www.facebook.com/groups/baanchiangmai/",
    "https://www.facebook.com/groups/694087674785430/",
    "https://www.facebook.com/groups/411301775702951/",
    "https://www.facebook.com/groups/169718747164928/",
    "https://www.facebook.com/groups/1475061816108017/",
    "https://www.facebook.com/groups/1582425938465943/",
    "https://www.facebook.com/groups/sale.rent.poolvillachiangmai/",
    "https://www.facebook.com/groups/251125079442673/",
    "https://www.facebook.com/groups/1034329704984830/",
    "https://www.facebook.com/groups/203683550205761/",
    "https://www.facebook.com/groups/1450131905304596/",
    "https://www.facebook.com/groups/959493160788393/",
    "https://www.facebook.com/groups/2897034136980512/",
    "https://www.facebook.com/groups/korn.property/",
    "https://www.facebook.com/groups/1456428424593312/",
    "https://www.facebook.com/groups/152080739566471/",
    "https://www.facebook.com/groups/Land.House.C.M.2014/",
    "https://www.facebook.com/groups/2336780789695894/",
    "https://www.facebook.com/groups/236116797208244/",
    "https://www.facebook.com/groups/landhomechiangmai/"
]

def login_to_facebook(driver):
    print("Login -> opening m.facebook.com")
    driver.get("https://m.facebook.com")
    time.sleep(3)
    for s in ["button[data-cookiebanner='accept_button_dialog']", "button[title='Allow all cookies']", "button[title='Accept All']", "button[aria-label='Allow all cookies']"]:
        btns = driver.find_elements(By.CSS_SELECTOR, s)
        if btns and btns[0].is_displayed():
            btns[0].click()
            time.sleep(2)
            break
    email = driver.find_elements(By.ID, "m_login_email")
    pwd = driver.find_elements(By.ID, "m_login_password")
    if not email:
        email = driver.find_elements(By.ID, "email")
        pwd = driver.find_elements(By.ID, "pass")
    if email and pwd:
        email[0].clear()
        email[0].send_keys(FACEBOOK_EMAIL)
        pwd[0].send_keys(FACEBOOK_PASSWORD)
        pwd[0].send_keys(Keys.RETURN)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[aria-label='Menu'], a[aria-label='Home'], a[aria-label='Search']")))
    print("Login -> completed")

def parse_relative_time_to_days(time_str):
    s = (time_str or "").strip().lower()
    if not s or "เมื่อสักครู่" in s or "just now" in s:
        return 0
    if "นาที" in s or "minute" in s or "min" in s or "ชม" in s or "ชั่วโมง" in s or "hour" in s or "hrs" in s or "hr" in s:
        return 0
    m = re.search(r'(\d+)', s)
    n = int(m.group(1)) if m else 0
    if "วัน" in s or "day" in s or "d" == s[-1:]:
        return n
    if "สัปดาห์" in s or "week" in s or "wk" in s or "w" == s[-1:]:
        return n * 7
    if "เดือน" in s or "month" in s or "mo" in s:
        return n * 30
    if "ปี" in s or "year" in s or "yr" in s:
        return n * 365
    return 999

def wait_present(driver, by, sel, timeout):
    t0 = time.time()
    while time.time() - t0 < timeout:
        if driver.find_elements(by, sel):
            return True
        time.sleep(0.25)
    return False

def page_loaded(driver):
    try_len = driver.execute_script("return document.body && document.body.innerHTML ? document.body.innerHTML.length : 0")
    return try_len and try_len > 1000

def human_reload_flow(driver, url):
    driver.get(url)
    ok = wait_present(driver, By.CSS_SELECTOR, "div[role='main'], #m_group_stories_container, article", 8)
    if ok or page_loaded(driver):
        return True
    driver.refresh()
    ok = wait_present(driver, By.CSS_SELECTOR, "div[role='main'], #m_group_stories_container, article", 8)
    if ok or page_loaded(driver):
        return True
    driver.get("https://www.livinginsider.com/")
    time.sleep(1.2)
    driver.get(url)
    ok = wait_present(driver, By.CSS_SELECTOR, "div[role='main'], #m_group_stories_container, article", 8)
    return ok or page_loaded(driver)

def normalize_group_url(u):
    v = u.strip().replace("://facebook.com", "://www.facebook.com").replace("://www.facebook.com", "://m.facebook.com")
    if "/members" in v:
        v = v.split("/members")[0] + "/"
    return v if v.endswith("/") else v + "/"

def collect_group_post_urls(driver, group_url):
    group_url = normalize_group_url(group_url)
    print(f"Group -> loading {group_url}")
    ok_nav = human_reload_flow(driver, group_url)
    if not ok_nav:
        print("Group -> load failed, skipping")
        return []
    seen = set()
    results = []
    prev_len = 0
    stagnant = 0
    loops = 0
    while True:
        data = driver.execute_script("""
            function absUrl(href){ if(!href) return null; if(href.indexOf('http')===0) return href.split('?')[0]; return location.origin + href.split('?')[0]; }
            var posts = Array.from(document.querySelectorAll('article, div[role="article"]'));
            var out = [];
            for (var i=0;i<posts.length;i++){
                var p = posts[i];
                var a = p.querySelector("a[href*='/posts/'], a[href*='/permalink/'], a[href*='story.php']");
                if(!a) continue;
                var href = absUrl(a.getAttribute('href'));
                if(!href) continue;
                var t = p.querySelector("a[aria-label], time[aria-label], abbr[aria-label]");
                var tlabel = null;
                if(t){ tlabel = t.getAttribute('aria-label') || t.textContent || ""; }
                if(!tlabel){
                    var t2 = p.querySelector("span[aria-label], div[aria-label]");
                    if(t2){ tlabel = t2.getAttribute('aria-label') || t2.textContent || ""; }
                }
                var ut = null;
                var uel = p.querySelector('abbr[data-utime], time[data-utime]');
                if(uel){ var v = parseInt(uel.getAttribute('data-utime')); if(!isNaN(v)) ut = v; }
                out.push([href, tlabel, ut]);
            }
            return out;
        """)
        for href, tlabel, ut in data:
            if href in seen:
                continue
            days_ago = parse_relative_time_to_days(tlabel)
            if (days_ago == 999) and ut:
                now_epoch = int(datetime.now(timezone.utc).timestamp())
                diff_days = int((now_epoch - int(ut)) // 86400)
                days_ago = diff_days if diff_days >= 0 else 999
            seen.add(href)
            if days_ago <= 7:
                results.append(href)
            elif days_ago > 7:
                return results
        driver.execute_script("window.scrollBy(0, 1400);")
        time.sleep(1.4)
        curr_len = len(seen)
        if curr_len == prev_len:
            stagnant += 1
        else:
            stagnant = 0
        prev_len = curr_len
        loops += 1
        if stagnant >= 6 or loops >= 3000:
            break
    print(f"Group -> collected {len(results)} URLs (<=7 days)")
    return results

def main():
    out_dir = Path(os.path.dirname(OUTPUT_CSV_FILE))
    if out_dir and not out_dir.exists():
        out_dir.mkdir(parents=True, exist_ok=True)
    print("Init -> creating Chrome profile")
    options = uc.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--lang=en-US")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument(f"--user-data-dir={PROFILE_PATH}")
    options.page_load_strategy = "eager"
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(90)
    driver.set_script_timeout(90)
    login_to_facebook(driver)
    print("Run -> collecting from all groups")
    rows = []
    total = 0
    for i, link in enumerate(GROUP_URLS, start=1):
        print(f"Group {i}/{len(GROUP_URLS)} -> starting")
        urls = collect_group_post_urls(driver, link)
        for u in urls:
            rows.append({'group_name': link.split("/groups/")[1].split("/")[0], 'group_url': link, 'post_url': u})
        total += len(urls)
        print(f"Group {i}/{len(GROUP_URLS)} -> total {total} URLs")
        time.sleep(1.0)
    driver.quit()
    print(f"Save -> writing CSV {OUTPUT_CSV_FILE} with {len(rows)} rows")
    df_out = pd.DataFrame(rows, columns=['group_name', 'group_url', 'post_url'])
    df_out.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print("Done")

if __name__ == "__main__":
    main()

Stage: Init -> สร้าง Chrome โปรไฟล์ถาวร
Stage: Init -> เรียก Login
Stage: Login -> เปิดหน้า m.facebook.com
Stage: Login -> สำเร็จ
Stage: Run -> เริ่มรวบรวมจากกลุ่มทั้งหมด
Stage: Group 1/44 -> เริ่ม https://www.facebook.com/groups/302468990428489/
Stage: Group -> เปิด https://m.facebook.com/groups/302468990428489/
Stage: Group -> ได้ 0 URL (<=7 วัน)
Stage: Group 1/44 -> สะสมทั้งหมด 0 URL
Stage: Group 2/44 -> เริ่ม https://www.facebook.com/groups/322977734828852/
Stage: Group -> เปิด https://m.facebook.com/groups/322977734828852/
Stage: Group -> ได้ 0 URL (<=7 วัน)
Stage: Group 2/44 -> สะสมทั้งหมด 0 URL
Stage: Group 3/44 -> เริ่ม https://www.facebook.com/groups/812156038944325/
Stage: Group -> เปิด https://m.facebook.com/groups/812156038944325/
Stage: Group -> ได้ 0 URL (<=7 วัน)
Stage: Group 3/44 -> สะสมทั้งหมด 0 URL
Stage: Group 4/44 -> เริ่ม https://www.facebook.com/groups/1472146056424210/
Stage: Group -> เปิด https://m.facebook.com/groups/1472146056424210/
Stage: Group -> ได้ 0 URL 

In [ ]:
import time
import os
import csv
from pathlib import Path
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv

load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
INPUT_LINKS_CSV = Path("data/Facebook_post_urls.csv")
OUTPUT_DETAILS_CSV = Path("data/facebook_full_content.csv")
PROFILE_PATH = Path("chrome-profile/fb-scraper")
WAIT = 30

def click_cookie(driver):
    for s in ['button[data-cookiebanner="accept_button"]', 'button[data-cookiebanner="accept_button_dialog"]', "button[title='Allow all cookies']", "button[aria-label='Allow all cookies']", "div[role='dialog'] div[aria-label*='cookies'] button"]:
        btns = driver.find_elements(By.CSS_SELECTOR, s)
        if btns and btns[0].is_displayed() and btns[0].is_enabled():
            btns[0].click()
            time.sleep(1)
            break

def login(driver):
    driver.get("https://www.facebook.com/?locale=en_US")
    time.sleep(2)
    click_cookie(driver)
    if "login" in driver.current_url.lower():
        WebDriverWait(driver, WAIT).until(EC.presence_of_element_located((By.ID, "email"))).send_keys(FACEBOOK_EMAIL)
        driver.find_element(By.ID, "pass").send_keys(FACEBOOK_PASSWORD)
        driver.find_element(By.NAME, "login").click()
    WebDriverWait(driver, WAIT).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]')))

def expand_see_more(driver, root):
    driver.execute_script("""
        (function(root){
          function clickAll(){
            var q = (root||document).querySelectorAll("div[role='button'],span[role='button'],a[role='button']");
            var did=false;
            for (var i=0;i<q.length;i++){
              var t=(q[i].innerText||"").trim();
              if(!t) continue;
              if(t.includes("See more")||t.includes("See More")||t.includes("ดูเพิ่มเติม")||t.includes("เพิ่มเติม")){
                q[i].click();did=true;
              }
            }
            return did;
          }
          var n=0; while(n<4 && clickAll()){ n++; }
        })(arguments[0]);
    """, root)

def extract_text(driver):
    txt = driver.execute_script("""
        function pick(){
          var sels=[
            "div[role='dialog'] [data-ad-rendering-role='story_message']",
            "div[role='main']  [data-ad-rendering-role='story_message']",
            "div[role='dialog'] [data-ad-preview='message']",
            "div[role='main']  [data-ad-preview='message']",
            "div[role='dialog'] article",
            "div[role='main']  article"
          ];
          for(var i=0;i<sels.length;i++){
            var e=document.querySelector(sels[i]);
            if(e){ return e; }
          }
          return document.querySelector("div[role='dialog']")||document.querySelector("div[role='main']");
        }
        var el=pick();
        return el? (el.textContent||"").trim() : "";
    """)
    if not txt:
        return ""
    for b in ["Like","Comment","Share","Send","Follow","Save","More","ถูกใจ","แสดงความคิดเห็น","แชร์","ส่งข้อความ","บันทึก","ติดตาม"]:
        txt = txt.replace(b, " ")
    return " ".join(txt.split()).strip()

def get_raw_post_text(driver, permalink):
    for step in range(3):
        driver.get(permalink)
        WebDriverWait(driver, WAIT).until(EC.any_of(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='dialog']")),
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='main']"))
        ))
        root = driver.find_elements(By.CSS_SELECTOR, "div[role='dialog'], div[role='main']")
        if root:
            expand_see_more(driver, root[0])
        text = extract_text(driver)
        if len(text) > 20:
            return text
        if step == 0:
            driver.refresh()
            time.sleep(2)
        else:
            driver.get("https://www.facebook.com/")
            time.sleep(1.5)
    return ""

if not FACEBOOK_EMAIL or not FACEBOOK_PASSWORD or not INPUT_LINKS_CSV.exists():
    raise SystemExit(1)

opts = uc.ChromeOptions()
opts.add_argument(f'--user-data-dir={PROFILE_PATH.as_posix()}')
opts.add_argument('--disable-notifications')
opts.add_argument('--lang=en-US')
opts.page_load_strategy = "eager"
driver = uc.Chrome(options=opts, version_main=142)
driver.set_page_load_timeout(90)
driver.set_script_timeout(90)

login(driver)

with open(INPUT_LINKS_CSV, 'r', encoding='utf-8-sig', newline='') as infile, open(OUTPUT_DETAILS_CSV, 'w', newline='', encoding='utf-8-sig') as outfile:
    r = csv.DictReader(infile)
    w = csv.writer(outfile)
    w.writerow(['post_url', 'content'])
    for row in r:
        url = (row.get('post_url') or '').strip()
        if not url:
            continue
        txt = get_raw_post_text(driver, url) or "N/A"
        w.writerow([url, txt])
        time.sleep(1.2)

driver.quit()

In [ ]:
import time
import csv
import re
from datetime import datetime, timedelta, timezone
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

START_URL = 'https://www.ddproperty.com/%E0%B8%A3%E0%B8%A7%E0%B8%A1%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A8%E0%B8%82%E0%B8%B2%E0%B8%A2?areaCode=&listingType=sale&regionCode=TH50&locale=th&slug=search&_freetextDisplay=%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88&isCommercial=false'
OUTPUT_CSV_FILE = 'data/ddproperty_listing_urls.csv'
WEBDRIVER_WAIT_TIMEOUT = 20
MAX_PAGES = 200

MONTH_ABBR = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'
DATE_REGEX = re.compile(rf'\b{MONTH_ABBR}\s+\d{{1,2}},\s+\d{{4}}\b', re.I)
REL_REGEX = re.compile(r'(\d+)\s*(h|d|w|mo|y)\s*ago', re.I)
TH_TZ = timezone(timedelta(hours=7))

def build_page_url(start_url, page):
    u = urlparse(start_url)
    q = urlencode(dict(parse_qsl(u.query)), doseq=True)
    path = u.path.rstrip('/')
    m = re.match(r'^(.*?)(/\d+)$', path)
    base = m.group(1) if m else path
    new_path = base if page <= 1 else base + f'/{page}'
    return urlunparse((u.scheme, u.netloc, new_path, u.params, q, u.fragment))

def within_30_days(text):
    now = datetime.now(TH_TZ)
    m = DATE_REGEX.search(text)
    if m:
        dt = datetime.strptime(m.group(0), "%b %d, %Y").date()
        delta = (now.date() - dt).days
        return 0 <= delta <= 30
    m2 = REL_REGEX.search(text)
    if m2:
        n = int(m2.group(1))
        unit = m2.group(2).lower()
        if unit == 'h':
            return True
        if unit == 'd':
            return n <= 30
        if unit == 'w':
            return n * 7 <= 30
        if unit == 'mo':
            return n * 30 <= 30
        if unit == 'y':
            return n * 365 <= 30
    return False

print("Init Chrome Driver...")
options = uc.ChromeOptions()
options.add_argument('--disable-notifications')
options.add_argument('--start-maximized')
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)

first_url = build_page_url(START_URL, 1)
print(f"Navigate to: {first_url}")
driver.get(first_url)

btns = driver.find_elements(By.XPATH, "//button[normalize-space(text())='ยอมรับ']")
if btns:
    btns[0].click()
    time.sleep(1)

all_listing_urls = set()
page_num = 1
last_first_href = ""

while page_num <= MAX_PAGES:
    print(f"Scraping page {page_num}")
    listing_sel = "a.listing-card-link, a.card-footer"
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, listing_sel)))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.4)

    cards = driver.find_elements(By.CSS_SELECTOR, listing_sel)
    hrefs_now = [e.get_attribute('href') for e in cards if e.get_attribute('href')]
    if not hrefs_now:
        print("No URLs found, stopping")
        break
    if page_num > 1 and hrefs_now[0] == last_first_href:
        print("Listing unchanged, stopping")
        break
    last_first_href = hrefs_now[0]

    hrefs_filtered = []
    for e in cards:
        h = e.get_attribute('href')
        if not h:
            continue
        rec_nodes = e.find_elements(By.XPATH, ".//span[@da-id='lc-feature-info' or contains(@class,'info-value') or (contains(@class,'pg-font-caption-xs') and contains(., 'โพสต์อีกครั้งเมื่อ'))]")
        if not rec_nodes:
            anc = None
            try:
                anc = e.find_element(By.XPATH, "./ancestor::div[contains(@class,'details-group-root') or contains(@class,'listing-card') or contains(@class,'content')][1]")
            except:
                pass
            if anc:
                rec_nodes = anc.find_elements(By.XPATH, ".//span[@da-id='lc-feature-info' or contains(@class,'info-value') or (contains(@class,'pg-font-caption-xs') and contains(., 'โพสต์อีกครั้งเมื่อ'))]")
        rec_text = rec_nodes[0].text if rec_nodes else e.text
        if within_30_days(rec_text):
            hrefs_filtered.append(h)

    new_urls = set(hrefs_filtered) - all_listing_urls
    print(f"Found {len(new_urls)} new URLs on page")
    all_listing_urls.update(hrefs_filtered)

    next_num = page_num + 1
    next_url = build_page_url(START_URL, next_num)
    prev_last = hrefs_now[-1]
    driver.get(next_url)

    t0 = time.time()
    changed = False
    while time.time() - t0 < WEBDRIVER_WAIT_TIMEOUT:
        state = driver.execute_script("return document.readyState")
        if state == "complete":
            els = driver.find_elements(By.CSS_SELECTOR, listing_sel)
            if els:
                h = els[0].get_attribute('href')
                hh = els[-1].get_attribute('href')
                if (h and h != last_first_href) or (hh and hh != prev_last):
                    changed = True
                    break
        time.sleep(0.3)
    if not changed:
        print("Next page unchanged, stopping")
        break

    page_num = next_num

driver.quit()

if all_listing_urls:
    print(f"Total URLs: {len(all_listing_urls)}")
    with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(['url'])
        for url in sorted(all_listing_urls):
            w.writerow([url])
    print("Process complete")

Initializing Chrome Driver...
Navigating to start URL: https://www.ddproperty.com/%E0%B8%A3%E0%B8%A7%E0%B8%A1%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A8%E0%B8%82%E0%B8%B2%E0%B8%A2?listingType=sale&regionCode=TH50&locale=th&slug=search&_freetextDisplay=%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88&isCommercial=false

--- Scraping Page 1 ---
Found 23 new listing URLs on this page.

--- Scraping Page 2 ---
Found 20 new listing URLs on this page.

--- Scraping Page 3 ---
Found 20 new listing URLs on this page.

--- Scraping Page 4 ---
Found 20 new listing URLs on this page.

--- Scraping Page 5 ---
Found 20 new listing URLs on this page.
Next page did not change within timeout. Ending pagination.

Total unique URLs collected: 103
Writing all URLs to ddproperty_listing_urls.csv...
Process complete.


In [ ]:
import time
import csv
from multiprocessing import Pool, cpu_count
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

INPUT_CSV_FILE = 'data/ddproperty_listing_urls.csv'
OUTPUT_CSV_FILE = 'data/ddproperty_scraped_details.csv'
WEBDRIVER_WAIT_TIMEOUT = 25
PROCESSES = max(1, min(cpu_count(), 4))

def text_or_empty(driver, css):
    els = driver.find_elements(By.CSS_SELECTOR, css)
    return els[0].text.strip() if els else ""

def first_el(driver, css):
    els = driver.find_elements(By.CSS_SELECTOR, css)
    return els[0] if els else None

def click_js(driver, el):
    driver.execute_script("arguments[0].click();", el)

def any_visible(driver, selector):
    return driver.execute_script("""
        const sel = arguments[0];
        const els = document.querySelectorAll(sel);
        for (const el of els) {
            const cs = getComputedStyle(el);
            const r = el.getBoundingClientRect();
            if (cs.display !== 'none' && cs.visibility !== 'hidden' && parseFloat(cs.opacity) !== 0 && r.width > 0 && r.height > 0) {
                return true;
            }
        }
        return false;
    """, selector)

def close_modal(driver, body_selectors):
    for sel in body_selectors:
        if any_visible(driver, sel):
            btn = first_el(driver, "button[da-id='modal-close-button'], button.btn-close, button[aria-label='Close']")
            if btn:
                click_js(driver, btn)
            t0 = time.time()
            while any_visible(driver, sel) and time.time() - t0 < WEBDRIVER_WAIT_TIMEOUT:
                time.sleep(0.2)
            return

def scrape_listing(driver, url):
    wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)
    details = {'URL': url}
    driver.get(url)
    btns = driver.find_elements(By.XPATH, "//button[normalize-space(text())='ยอมรับ']")
    if btns:
        click_js(driver, btns[0])
        time.sleep(1)
    main_sel = "div[da-id='property-snapshot-info']"
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, main_sel)))
    details['Title'] = text_or_empty(driver, "h1[da-id='property-title']")
    details['Address'] = text_or_empty(driver, "p[da-id='property-address']")
    price_txt = text_or_empty(driver, "h2[da-id='price-amount']")
    details['Price'] = price_txt.replace('฿', '').strip()
    for a in driver.find_elements(By.CSS_SELECTOR, "div.amenity[da-id*='-amenity']"):
        labels = [i.get_attribute('aria-label') for i in a.find_elements(By.TAG_NAME, 'img') if i.get_attribute('aria-label')]
        val_el = a.find_elements(By.CSS_SELECTOR, "p.amenity-text")
        if labels and val_el:
            details[labels[0]] = val_el[0].text.strip()
    see_more_btn = first_el(driver, "button[da-id='meta-table-see-more-btn'], button[da-id='see-more-meta']")
    if see_more_btn:
        click_js(driver, see_more_btn)
        modal_sel = "div.property-modal-body"
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, modal_sel)))
        modal = first_el(driver, modal_sel)
        if modal:
            for w in modal.find_elements(By.CSS_SELECTOR, "div.property-modal-body-wrapper"):
                alts = [i.get_attribute('alt') for i in w.find_elements(By.TAG_NAME, 'img') if i.get_attribute('alt')]
                val_el = w.find_elements(By.CSS_SELECTOR, "p.property-modal-body-value")
                if alts and val_el:
                    key = alts[0].replace('-o', '').replace('-', ' ').title()
                    details[key] = val_el[0].text.strip()
        close_modal(driver, [modal_sel])
    read_btn = first_el(driver, "button[da-id='description-widget-show-more-lnk'], button[da-id='property-description-show-more']")
    if read_btn:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", read_btn)
        time.sleep(0.5)
        click_js(driver, read_btn)
        desc_modal_sels = ["div.description-modal-body", "div[da-id='description-modal-body']"]
        expanded_sels = ["div[da-id='description-widget-body']", "div[da-id='description-text']", "div.property-description"]
        desc_body = None
        t0 = time.time()
        while time.time() - t0 < WEBDRIVER_WAIT_TIMEOUT:
            for sel in desc_modal_sels:
                el = first_el(driver, sel)
                if el and any_visible(driver, sel):
                    desc_body = el
                    break
            if desc_body:
                break
            for sel in expanded_sels:
                el = first_el(driver, sel)
                if el and el.text.strip():
                    details['Description'] = el.text.strip()
                    desc_body = None
                    break
            if 'Description' in details:
                break
            time.sleep(0.2)
        if desc_body:
            html = desc_body.get_attribute('innerHTML') or ""
            details['Description'] = html.replace('<br>', '\n').strip()
            close_modal(driver, desc_modal_sels)
        if 'Description' not in details:
            details['Description'] = text_or_empty(driver, "div[da-id='description-widget-body']")
    return details

def run_batch(urls):
    options = uc.ChromeOptions()
    options.add_argument('--disable-notifications')
    options.add_argument('--start-maximized')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--no-sandbox')
    options.page_load_strategy = 'eager'
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(180)
    out = []
    for url in urls:
        out.append(scrape_listing(driver, url))
        time.sleep(2.0)
    driver.quit()
    return out

def chunk(seq, size):
    return [seq[i:i+size] for i in range(0, len(seq), size)]

with open(INPUT_CSV_FILE, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    urls_to_scrape = [row[0] for row in reader if row and row[0].strip()]

if not urls_to_scrape:
    with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
        csv.DictWriter(f, fieldnames=['URL']).writeheader()
else:
    batch_size = max(1, (len(urls_to_scrape) + PROCESSES - 1) // PROCESSES)
    with Pool(PROCESSES) as pool:
        results = pool.map(run_batch, chunk(urls_to_scrape, batch_size))
    all_details = [d for batch in results for d in batch]
    headers = sorted({"URL"} | {k for d in all_details for k in d.keys()})
    with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
        w = csv.DictWriter(f, fieldnames=headers)
        w.writeheader()
        w.writerows(all_details)

In [ ]:
import time
import csv
import re
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

START_URL = 'https://www.livinginsider.com/living_zone/45/all/all/1/%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88.html'
OUTPUT_CSV_FILE = 'data/livinginsider_listing_urls.csv'
PAGE_TIMEOUT = 45
MAX_PAGES = 200

def build_page_url(u, page):
    p = urlparse(u)
    parts = [x for x in p.path.split('/') if x]
    k = None
    for i, seg in enumerate(parts):
        if re.fullmatch(r'\d+', seg):
            k = i
    if k is None:
        base = p.path.rstrip('/')
        new_path = base if page <= 1 else base + f'/{page}'
    else:
        parts[k] = '1' if page <= 1 else str(page)
        new_path = '/' + '/'.join(parts)
    return urlunparse((p.scheme, p.netloc, new_path, p.params, urlencode(dict(parse_qsl(p.query)), doseq=True), p.fragment))

def wait_ready(driver, timeout):
    t0 = time.time()
    while time.time() - t0 < timeout:
        if driver.execute_script("return document.readyState") == "complete":
            return True
        time.sleep(0.2)
    return False

def deep_scroll(driver, rounds=18, pause=0.7):
    h0 = 0
    for _ in range(rounds):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        h = driver.execute_script("return document.body.scrollHeight")
        if h == h0:
            break
        h0 = h

def collect_links_js(driver, base):
    hrefs = driver.execute_script("""
const out = [];
const a1 = Array.from(document.querySelectorAll("a[href*='/livingdetail/'][href$='.html']"));
const a2 = Array.from(document.querySelectorAll("a[href^='/livingdetail/'][href$='.html']"));
const all = [...a1, ...a2];
for (const a of all){
  const h = a.getAttribute('href') || '';
  if (h && !h.includes('bclick') && !h.includes('stories') && !h.includes('banner')){
    out.push(h);
  }
}
return Array.from(new Set(out));
""")
    fixed = []
    for h in hrefs:
        fixed.append(base + h if h.startswith("/") else h)
    return list(dict.fromkeys(fixed))

options = uc.ChromeOptions()
options.add_argument('--disable-notifications')
options.add_argument('--start-maximized')
driver = uc.Chrome(options=options)
base = f"{urlparse(START_URL).scheme}://{urlparse(START_URL).netloc}"

page = 1
all_urls = set()
last_first = ""
driver.get(build_page_url(START_URL, page))
wait_ready(driver, PAGE_TIMEOUT)

while page <= MAX_PAGES:
    deep_scroll(driver, rounds=20, pause=0.8)
    urls_now = collect_links_js(driver, base)
    if not urls_now:
        break
    if page > 1 and urls_now[0] == last_first:
        break
    last_first = urls_now[0]
    all_urls.update(urls_now)

    page += 1
    prev_last = urls_now[-1]
    driver.get(build_page_url(START_URL, page))
    t0 = time.time()
    changed = False
    while time.time() - t0 < PAGE_TIMEOUT:
        wait_ready(driver, 3)
        deep_scroll(driver, rounds=6, pause=0.6)
        cur = collect_links_js(driver, base)
        if cur and (cur[0] != last_first or cur[-1] != prev_last):
            changed = True
            break
        time.sleep(0.3)
    if not changed:
        break

driver.quit()

if all_urls:
    with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(['url'])
        for u in sorted(all_urls):
            w.writerow([u])

In [ ]:
import re
import csv
import sys
import time
from pathlib import Path
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

INPUT_CSV = Path("data/livinginsider_listing_urls.csv")
OUTPUT_CSV = "data/livinginsider_scraped_details.csv"

def first_text(elems):
    return elems[0].text.strip() if elems else ""

if not INPUT_CSV.exists():
    hits = list(Path.cwd().rglob("livinginsider_listing_urls.csv"))
    INPUT_CSV = hits[0] if hits else None
    if not INPUT_CSV:
        sys.exit(1)

def click_consent(driver):
    btns = driver.find_elements(By.CSS_SELECTOR, "#onetrust-accept-btn-handler")
    btns += driver.find_elements(By.XPATH, "//*[self::a or self::button][contains(., 'ยอมรับ')]")
    btns = [e for e in btns if e.is_displayed() and e.is_enabled()]
    if btns:
        driver.execute_script("arguments[0].click();", btns[0])
        time.sleep(0.3)

def wait_present(driver, by, sel, timeout):
    t0 = time.time()
    while time.time() - t0 < timeout:
        if driver.find_elements(by, sel):
            return True
        time.sleep(0.25)
    return False

def page_loaded(driver):
    try_len = driver.execute_script("return document.body && document.body.innerHTML ? document.body.innerHTML.length : 0")
    return try_len and try_len > 1000

def human_reload(driver, url):
    driver.get(url)
    click_consent(driver)
    ok = wait_present(driver, By.CSS_SELECTOR, "div.space_padding_top_data", 8)
    if ok or page_loaded(driver):
        return True
    driver.refresh()
    click_consent(driver)
    ok = wait_present(driver, By.CSS_SELECTOR, "div.space_padding_top_data", 8)
    if ok or page_loaded(driver):
        return True
    driver.get("https://www.livinginsider.com/")
    time.sleep(1.2)
    driver.get(url)
    click_consent(driver)
    ok = wait_present(driver, By.CSS_SELECTOR, "div.space_padding_top_data", 8)
    return ok or page_loaded(driver)

def parse_coords(text):
    m = re.search(r"(-?\d+(?:\.\d+)?)\s*,\s*(-?\d+(?:\.\d+)?)", text)
    return f"{m.group(1)},{m.group(2)}" if m else ""

def scrape_one(driver, url):
    ok = human_reload(driver, url)
    if not ok:
        return None
    desc = first_text(driver.find_elements(By.CSS_SELECTOR, "#desc-text-nl .wordwrap-box .wordwrap"))
    phone_nodes = driver.find_elements(By.CSS_SELECTOR, "a.p-phone-contact[data-zcgrbcb]")
    phones = ", ".join(sorted({n.get_attribute("data-zcgrbcb") for n in phone_nodes if n.get_attribute("data-zcgrbcb")}))
    vc = [e.text.strip() for e in driver.find_elements(By.CSS_SELECTOR, ".box-show-view-click .text-custom-gray-new")]
    return {
        "URL": url,
        "Title": first_text(driver.find_elements(By.CSS_SELECTOR, "h1.font_sarabun.show-title")),
        "Price": first_text(driver.find_elements(By.CSS_SELECTOR, ".show_price_topic .price_topic b")),
        "PricePerArea": first_text(driver.find_elements(By.CSS_SELECTOR, ".show_price_topic .price_cal_area_text")),
        "PropertyType": first_text(driver.find_elements(By.CSS_SELECTOR, ".box_tag_topic_detail .box_tag_building")),
        "PostType": first_text(driver.find_elements(By.CSS_SELECTOR, ".box_tag_topic_detail .box_tag_posttype")),
        "Size": first_text(driver.find_elements(By.CSS_SELECTOR, ".detail_property_list_des_new .detail-property-list-text")),
        "Description": desc,
        "CreatedDate": first_text(driver.find_elements(By.CSS_SELECTOR, ".row-detail-time .font_10_date")),
        "Boosted": first_text(driver.find_elements(By.XPATH, "//div[contains(@class,'row-detail-time')]//span[contains(text(),'ดันประกาศล่าสุด')]")),
        "Location": first_text(driver.find_elements(By.CSS_SELECTOR, ".form-group.group-location-detail .detail-text-zone a")),
        "Views": vc[0] if len(vc) > 0 else "",
        "Clicks": vc[1] if len(vc) > 1 else "",
        "Coordinates": parse_coords(desc),
        "MaskedContacts": phones
    }

options = uc.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.page_load_strategy = "eager"
driver = uc.Chrome(options=options)

with open(str(INPUT_CSV), "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader, None)
    urls = [r[0].strip() for r in reader if r and r[0].strip()]

rows = [r for u in urls if (r := scrape_one(driver, u)) for _ in [time.sleep(0.4)]]

driver.quit()

with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=["URL","Title","Price","PricePerArea","PropertyType","PostType","Size","Description","CreatedDate","Boosted","Location","Views","Clicks","Coordinates","MaskedContacts"])
    w.writeheader()
    for r in rows:
        w.writerow(r)

In [ ]:
import time
import csv
import re
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

START_URL = "https://baan.kaidee.com/c2p1-realestate/chiangmai"
OUTPUT_CSV_FILE = "data/kaidee_listing_urls.csv"
PAGE_TIMEOUT = 40
MAX_PAGES = 200
PATTERN = re.compile(r"https://baan\.kaidee\.com/product-\d+")

def build_page_url(u, page):
    if page <= 1:
        return u
    return (u if u.endswith("/") else u + "/") + f"p-{page}"

def wait_ready(driver, timeout):
    t0 = time.time()
    while time.time() - t0 < timeout:
        if driver.execute_script("return document.readyState") == "complete":
            return True
        time.sleep(0.2)
    return False

def deep_scroll(driver, rounds=12, pause=0.8):
    prev = 0
    for _ in range(rounds):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        cur = driver.execute_script("return document.body.scrollHeight")
        if cur == prev:
            break
        prev = cur

def extract_links(html):
    return sorted(set(PATTERN.findall(html)))

options = uc.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)

page = 1
all_urls = set()
last_first = ""

while page <= MAX_PAGES:
    driver.get(build_page_url(START_URL, page))
    wait_ready(driver, PAGE_TIMEOUT)
    deep_scroll(driver, rounds=15, pause=0.8)
    links = extract_links(driver.page_source)
    if not links:
        break
    if page > 1 and links[0] == last_first:
        break
    last_first = links[0]
    all_urls.update(links)
    page += 1

driver.quit()

if all_urls:
    with open(OUTPUT_CSV_FILE, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["url"])
        for u in sorted(all_urls):
            w.writerow([u])

In [ ]:
import csv
import time
from pathlib import Path
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

OUTPUT_CSV_FILE = "data/kaidee_scraped_details.csv"
WAIT = 25

cands = [
    Path("data/kaidee_listing_urls.csv"),
    Path("kaidee_listing_urls.csv")
]
INPUT_CSV = None
for p in cands:
    if p.exists():
        INPUT_CSV = p
        break
if not INPUT_CSV:
    for p in Path.cwd().rglob("kaidee_listing_urls.csv"):
        INPUT_CSV = p
        break

def scrape(driver, url):
    w = WebDriverWait(driver, WAIT)
    driver.get(url)
    btns = driver.find_elements(By.CSS_SELECTOR, "button[aria-label*='cookie i understand'], button[aria-label*='accept'], button:has(span[lang])")
    if btns:
        driver.execute_script("arguments[0].click();", btns[0])
        time.sleep(0.3)
    title_el = w.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.sc-747m9u-7")))
    price_el = w.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-1w68tq4-0 span.sc-3tpgds-0")))
    desc_el = driver.find_elements(By.CSS_SELECTOR, "div.sc-1kndlp1-0 p.inner-text")
    area_el = driver.find_elements(By.XPATH, "//ul[@id='has-attributes']//li[.//span[text()='เนื้อที่']]//span//b")
    phone_els = driver.find_elements(By.CSS_SELECTOR, "span.masked[data-value]")
    phones = sorted({e.get_attribute("data-value").strip() for e in phone_els if e.get_attribute("data-value")})
    return {
        "URL": url,
        "Title": title_el.text.strip(),
        "Price": price_el.text.strip(),
        "Area": area_el[0].text.strip() if area_el else "",
        "Phone": ", ".join(phones),
        "Description": desc_el[0].text.strip() if desc_el else ""
    }

if INPUT_CSV:
    options = uc.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--start-maximized")
    driver = uc.Chrome(options=options, version_main=140)
    with open(INPUT_CSV, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader, None)
        urls = [r[0].strip() for r in reader if r and r[0].strip()]
    rows = []
    for u in urls:
        rows.append(scrape(driver, u))
        time.sleep(1)
    driver.quit()
    with open(OUTPUT_CSV_FILE, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["URL", "Title", "Price", "Area", "Phone", "Description"])
        w.writeheader()
        w.writerows(rows)

In [ ]:
import time
import os
import csv
import re
from dotenv import load_dotenv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
START_URL = 'https://www.facebook.com/marketplace/chiangmai/propertyrentals/?exact=false'
OUTPUT_CSV_FILE = 'data/facebook_marketplace_urls.csv'
TARGET_URL_COUNT = 500
WEBDRIVER_WAIT_TIMEOUT = 20

def normalize_marketplace_url(u):
    if not u:
        return None
    u = u.strip().split('?')[0].split('#')[0]
    m = re.search(r'/marketplace/item/(\d+)', u)
    return f'https://www.facebook.com/marketplace/item/{m.group(1)}/' if m else None

options = uc.ChromeOptions()
options.add_argument('--disable-notifications')
options.add_argument('--start-maximized')
options.add_argument(f'--user-data-dir=chrome-profile/fb-marketplace')

driver = uc.Chrome(options=options, version_main=140)
wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)

driver.get(START_URL)
time.sleep(7)

popup_container_selector = "div[aria-label='Accessible login form']"
popup_forms = driver.find_elements(By.CSS_SELECTOR, popup_container_selector)

if popup_forms:
    popup_form = popup_forms[0]
    email_input = popup_form.find_element(By.NAME, "email")
    pass_input = popup_form.find_element(By.NAME, "pass")
    login_button = popup_form.find_element(By.CSS_SELECTOR, "div[aria-label='Log in to Facebook']")
    email_input.send_keys(FACEBOOK_EMAIL)
    pass_input.send_keys(FACEBOOK_PASSWORD)
    login_button.click()
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, popup_container_selector)))
    time.sleep(5)

all_listing_urls = set()
stagnant_scrolls = 0

while len(all_listing_urls) < TARGET_URL_COUNT:
    last_height = driver.execute_script("return document.body.scrollHeight")
    item_link_selector = "a[href*='/marketplace/item/']"
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, item_link_selector)))
    for element in driver.find_elements(By.CSS_SELECTOR, item_link_selector):
        norm = normalize_marketplace_url(element.get_attribute('href'))
        if norm:
            all_listing_urls.add(norm)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        stagnant_scrolls += 1
    else:
        stagnant_scrolls = 0
    if stagnant_scrolls >= 5:
        break

driver.quit()

if all_listing_urls:
    with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(['url'])
        for url in sorted(list(all_listing_urls))[:TARGET_URL_COUNT]:
            w.writerow([url])

In [ ]:
import time
import os
import csv
from pathlib import Path
from dotenv import load_dotenv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
INPUT_CSV_FILE = Path("data/facebook_marketplace_urls.csv")
OUTPUT_CSV_FILE = Path("data/facebook_marketplace_details.csv")
WEBDRIVER_WAIT_TIMEOUT = 25
SCROLL_STEP_PX = 900
MAX_SCROLL_TRIES = 15

def scrape_marketplace_details(driver, url):
    wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)
    details = {'URL': url}
    driver.get(url)
    h1 = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))
    details['Title'] = h1.text.strip()
    price_xpath = "//h1/following-sibling::div[1]//span"
    details['Price'] = driver.find_element(By.XPATH, price_xpath).text.strip()
    detail_items = driver.find_elements(By.CSS_SELECTOR, "div[role='listitem'] span")
    details['Property_Details'] = ' | '.join([item.text.strip() for item in detail_items if item.text.strip()])
    
    desc_h2_xpath = "//h2[.//span[contains(., 'คำอธิบาย')] or .//span[contains(., 'Description')]]"
    desc_h2_elements = driver.find_elements(By.XPATH, desc_h2_xpath)
    if not desc_h2_elements:
        last_height = 0
        for i in range(MAX_SCROLL_TRIES):
            driver.execute_script(f"window.scrollBy(0, {SCROLL_STEP_PX});")
            time.sleep(0.8)
            desc_h2_elements = driver.find_elements(By.XPATH, desc_h2_xpath)
            if desc_h2_elements:
                break
            new_height = driver.execute_script("return document.body.scrollHeight;")
            if new_height == last_height:
                driver.execute_script("window.scrollTo(0, Math.max(document.documentElement.scrollTop - 600, 0));")
                time.sleep(0.5)
            last_height = new_height
    
    description_text = ""
    if desc_h2_elements:
        desc_h2 = desc_h2_elements[0]
        driver.execute_script("arguments[0].scrollIntoView({behavior:'instant', block:'center'});", desc_h2)
        time.sleep(0.6)
        containers = desc_h2.find_elements(By.XPATH, "./following-sibling::div | parent::div/following-sibling::div | ancestor::div[1]/following-sibling::div")
        container = containers[0] if containers else None
        if not container:
            cands = driver.find_elements(By.XPATH, "//h2[.//span[contains(., 'คำอธิบาย') or contains(., 'Description')]]/following::div[1]")
            container = cands[0] if cands else None
        if container:
            for _ in range(3):
                see_more = container.find_elements(By.XPATH, ".//div[@role='button'][.//span[contains(., 'ดูเพิ่มเติม') or contains(., 'See more')]]")
                if see_more:
                    if ("ดูน้อยลง" in see_more[0].text) or ("See less" in see_more[0].text):
                        break
                    driver.execute_script("arguments[0].click();", see_more[0])
                    time.sleep(0.6)
                else:
                    alt = desc_h2.find_elements(By.XPATH, "./following-sibling::div//div[@role='button'][.//span[contains(., 'ดูเพิ่มเติม') or contains(., 'See more')]]")
                    if alt:
                        driver.execute_script("arguments[0].click();", alt[0])
                        time.sleep(0.6)
                    else:
                        break
            spans = container.find_elements(By.XPATH, ".//span[string-length(normalize-space())>0]")
            parts = [s.text.strip() for s in spans if s.text.strip()]
            description_text = "\n".join(parts) if parts else (container.get_attribute("innerText") or "").strip()
    details['Description'] = description_text
    return details

options = uc.ChromeOptions()
options.add_argument('--disable-notifications')
options.add_argument('--start-maximized')
options.add_argument('--no-first-run')
options.add_argument('--no-default-browser-check')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--lang=en-US')
options.add_argument('--user-data-dir=chrome-profile/fb-marketplace')
options.page_load_strategy = 'eager'
driver = uc.Chrome(options=options)
driver.command_executor._client_config.timeout = 300
driver.set_page_load_timeout(240)
driver.set_script_timeout(240)

wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)
driver.get("https://www.facebook.com/?locale=en_US")
time.sleep(7)

popup_container_selector = "div[aria-label='Accessible login form'], div[aria-label='เข้าสู่ระบบแบบช่วยการเข้าถึง']"
popup_forms = driver.find_elements(By.CSS_SELECTOR, popup_container_selector)
if popup_forms:
    popup_form = popup_forms[0]
    email_input = popup_form.find_element(By.NAME, "email")
    pass_input = popup_form.find_element(By.NAME, "pass")
    login_button = popup_form.find_element(By.CSS_SELECTOR, "div[aria-label*='Log in'], div[aria-label*='เข้าสู่ระบบ']")
    email_input.clear()
    email_input.send_keys(FACEBOOK_EMAIL)
    pass_input.clear()
    pass_input.send_keys(FACEBOOK_PASSWORD)
    login_button.click()
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, popup_container_selector)))
    time.sleep(5)

with open(INPUT_CSV_FILE, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    urls_to_scrape = [row[0].strip() for row in reader if row and row[0].strip()]

all_details = []
for url in urls_to_scrape:
    all_details.append(scrape_marketplace_details(driver, url))
    time.sleep(2.5)

driver.quit()

if all_details:
    headers = ['URL', 'Title', 'Price', 'Property_Details', 'Description']
    with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
        w = csv.DictWriter(f, fieldnames=headers)
        w.writeheader()
        w.writerows(all_details)